Qdrant

In [ ]:
# # Install Qdrant client
# !pip install qdrant-client -q

# from qdrant_client import QdrantClient
# from qdrant_client.models import Distance, VectorParams
# import getpass

# # 🔑 Secure API Key Input (invisible)
# print("🔑 Enter your Qdrant API Key (input will be hidden):")
# qdrant_api_key = getpass.getpass("Qdrant API Key: ")

# # Verify key format
# if qdrant_api_key and len(qdrant_api_key) > 10:
#     print("✅ API Key captured securely")
# else:
#     print("⚠️ API Key seems invalid")

# # Connect to your cluster
# client = QdrantClient(
#     url="https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io",
#     api_key=qdrant_api_key
# )

# # Create collection
# client.create_collection(
#     collection_name="clinical_trials",
#     vectors_config=VectorParams(size=384, distance=Distance.COSINE)
# )

# print("✅ Collection 'clinical_trials' created successfully!")

# # Verify
# collections = client.get_collections()
# print(f"\n📊 Collections: {collections}")


🔑 Enter your Qdrant API Key (input will be hidden):
Qdrant API Key: ··········
✅ API Key captured securely


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `clinical_trials` already exists!"},"time":0.038894767}'

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# import os

# BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# # Check if files exist
# files_to_check = [
#     "clinical_trials_all_full_embeddings.npy",
#     "clinical_trials_all_full_chunk_map.json",
#     "clinical_trials_all_full_faiss.index"
# ]

# print("📁 Checking if files exist in Drive:\n")
# for filename in files_to_check:
#     filepath = f"{BASE}/{filename}"
#     exists = os.path.exists(filepath)
#     if exists:
#         size_mb = os.path.getsize(filepath) / (1024*1024)
#         print(f"✅ {filename}: EXISTS ({size_mb:.1f} MB)")
#     else:
#         print(f"❌ {filename}: NOT FOUND")


📁 Checking if files exist in Drive:

✅ clinical_trials_all_full_embeddings.npy: EXISTS (384.8 MB)
✅ clinical_trials_all_full_chunk_map.json: EXISTS (230.6 MB)
✅ clinical_trials_all_full_faiss.index: EXISTS (384.8 MB)


Load Data and Upload to Qdrant

In [ ]:
# import pandas as pd
# import numpy as np
# import json
# from tqdm import tqdm

# BASE = "/content/drive/.shortcut-targets-by-id/1-SiVJhXHTHtDYSrPmW_0VfuP7gSTePcj/data"

# print("⏳ Loading embeddings and chunk map from Drive...")

# # Load embeddings (384.8 MB)
# embeddings = np.load(f"{BASE}/clinical_trials_all_full_embeddings.npy")
# print(f"✅ Loaded {len(embeddings)} embeddings (shape: {embeddings.shape})")

# # Load chunk map (metadata - 230.6 MB)
# with open(f"{BASE}/clinical_trials_all_full_chunk_map.json", "r") as f:
#     chunk_map = json.load(f)
# print(f"✅ Loaded {len(chunk_map)} chunks of metadata")

# # Verify sizes match
# if len(embeddings) == len(chunk_map):
#     print(f"✅ Data verified: {len(embeddings)} vectors ready to upload")
# else:
#     print(f"⚠️ WARNING: Mismatch! {len(embeddings)} embeddings vs {len(chunk_map)} chunks")


⏳ Loading embeddings and chunk map from Drive...
✅ Loaded 262660 embeddings (shape: (262660, 384))
✅ Loaded 262660 chunks of metadata
✅ Data verified: 262660 vectors ready to upload


In [ ]:
# from qdrant_client.models import PointStruct
# import getpass

# # Reconnect to Qdrant (in case session expired)
# print("\n🔑 Enter your Qdrant API Key again:")
# qdrant_api_key = getpass.getpass("Qdrant API Key: ")

# from qdrant_client import QdrantClient

# client = QdrantClient(
#     url="https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io",
#     api_key=qdrant_api_key
# )
# print("✅ Connected to Qdrant")

# print("\n⏳ Uploading vectors to Qdrant...")
# print("⚠️ This will take 5-10 minutes for 262K vectors. Please wait...\n")

# # Batch upload (100 vectors at a time)
# batch_size = 100
# total_batches = (len(embeddings) + batch_size - 1) // batch_size

# for i in tqdm(range(0, len(embeddings), batch_size), desc="Uploading", total=total_batches):
#     batch_end = min(i + batch_size, len(embeddings))

#     points = []
#     for idx in range(i, batch_end):
#         points.append(
#             PointStruct(
#                 id=idx,
#                 vector=embeddings[idx].tolist(),
#                 payload={
#                     "nct_id": chunk_map[idx]["nct_id"],
#                     "title": chunk_map[idx]["title"],
#                     "text": chunk_map[idx]["text"],
#                     "status": chunk_map[idx]["status"]
#                 }
#             )
#         )

#     # Upload batch
#     client.upsert(
#         collection_name="clinical_trials",
#         points=points
#     )

# print(f"\n✅ Successfully uploaded {len(embeddings)} vectors to Qdrant!")

# # Verify upload
# collection_info = client.get_collection("clinical_trials")
# print(f"\n📊 Final Collection Stats:")
# print(f"   ✅ Total vectors: {collection_info.points_count:,}")
# print(f"   ✅ Vector dimension: {collection_info.config.params.vectors.size}")
# print(f"   ✅ Distance metric: {collection_info.config.params.vectors.distance}")



🔑 Enter your Qdrant API Key again:
Qdrant API Key: ··········
✅ Connected to Qdrant

⏳ Uploading vectors to Qdrant...
⚠️ This will take 5-10 minutes for 262K vectors. Please wait...



Uploading: 100%|██████████| 2627/2627 [13:20<00:00,  3.28it/s]


✅ Successfully uploaded 262660 vectors to Qdrant!

📊 Final Collection Stats:
   ✅ Total vectors: 262,660
   ✅ Vector dimension: 384
   ✅ Distance metric: Cosine


Data in Qdrant

In [1]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile update_qdrant_auto.py
"""
Automatically finds ALL CSV files in Drive folder and uploads to Qdrant
No manual file listing needed!
"""

import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, Distance, VectorParams
from tqdm import tqdm
import os
import glob

class QdrantAutoPipeline:
    def __init__(self, qdrant_url, qdrant_api_key):
        self.client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)
        self.embed_model = SentenceTransformer("all-MiniLM-L6-v2")
        self.collection_name = "clinical_trials"

    def find_all_csv_files(self, drive_folder_path):
        """Automatically find all CSV files in Drive folder"""
        print(f"🔍 Searching for CSV files in: {drive_folder_path}")

        # Find all CSV files
        csv_files = glob.glob(f"{drive_folder_path}/*.csv")

        if not csv_files:
            print("❌ No CSV files found!")
            return []

        print(f"✅ Found {len(csv_files)} CSV files:")
        for csv_file in csv_files:
            filename = os.path.basename(csv_file)
            size_mb = os.path.getsize(csv_file) / (1024 * 1024)
            print(f"   - {filename} ({size_mb:.1f} MB)")

        return csv_files

    def load_and_filter_csvs(self, csv_files):
        """Load all CSV files and filter"""
        print("\n📂 Loading CSV files...")

        dfs = []
        for csv_path in csv_files:
            filename = os.path.basename(csv_path)
            print(f"   Loading {filename}...")
            try:
                df = pd.read_csv(csv_path)
                dfs.append(df)
                print(f"      ✅ {len(df)} rows")
            except Exception as e:
                print(f"      ⚠️ Error loading {filename}: {e}")
                continue

        if not dfs:
            print("❌ No data loaded!")
            return None

        # Concatenate all
        df_all = pd.concat(dfs, ignore_index=True)
        print(f"\n✅ Total trials loaded: {len(df_all):,}")

        # Remove duplicates by NCT ID
        initial_count = len(df_all)
        df_all = df_all.drop_duplicates(subset=['nct_id'], keep='first')
        duplicates_removed = initial_count - len(df_all)
        if duplicates_removed > 0:
            print(f"🗑️ Removed {duplicates_removed:,} duplicate trials")

        # Filter bad statuses
        df_all["status"] = df_all["status"].astype(str).str.strip().str.title()
        bad_status = ["Terminated", "Withdrawn", "Suspended", "No Longer Available", "Unknown"]
        df_clean = df_all[~df_all["status"].isin(bad_status)].copy()

        filtered_out = len(df_all) - len(df_clean)
        print(f"🗑️ Filtered out {filtered_out:,} trials with bad status")
        print(f"✅ Final clean dataset: {len(df_clean):,} trials")

        return df_clean

    def create_chunks(self, df_clean):
        """Create text chunks from DataFrame"""
        print("\n📝 Creating chunks...")

        chunks = []
        skipped = 0

        for idx, row in tqdm(df_clean.iterrows(), total=len(df_clean), desc="Processing"):
            title = str(row.get("brief_title", "")).strip()
            summary = str(row.get("brief_summary", "")).strip()

            if len(summary) < 20:
                skipped += 1
                continue

            text = f"Title: {title}\nSummary: {summary}"

            chunks.append({
                "nct_id": row["nct_id"],
                "title": title,
                "text": text,
                "status": row["status"]
            })

        if skipped > 0:
            print(f"⚠️ Skipped {skipped:,} trials with insufficient summary")
        print(f"✅ Created {len(chunks):,} chunks")

        return chunks

    def generate_embeddings(self, chunks):
        """Generate embeddings for all chunks"""
        print("\n🧠 Generating embeddings...")
        print("⏳ This may take several minutes for large datasets...")

        texts = [c["text"] for c in chunks]
        embeddings = self.embed_model.encode(
            texts,
            batch_size=64,
            show_progress_bar=True,
            convert_to_numpy=True
        )

        print(f"✅ Generated {len(embeddings):,} embeddings (shape: {embeddings.shape})")
        return embeddings

    def upload_to_qdrant(self, embeddings, chunks, mode="refresh"):
        """Upload data to Qdrant"""

        if mode == "refresh":
            print("\n🗑️ Deleting old collection...")
            try:
                self.client.delete_collection(self.collection_name)
                print("✅ Old collection deleted")
            except:
                print("⚠️ No existing collection to delete")

            print("📦 Creating fresh collection...")
            self.client.create_collection(
                collection_name=self.collection_name,
                vectors_config=VectorParams(size=384, distance=Distance.COSINE)
            )
            print("✅ Collection created")
            start_id = 0
        else:  # mode == "add"
            collection_info = self.client.get_collection(self.collection_name)
            start_id = collection_info.points_count
            print(f"\n📊 Adding to existing data, starting from ID: {start_id:,}")

        print(f"\n⏳ Uploading {len(embeddings):,} vectors to Qdrant...")

        batch_size = 100
        total_batches = (len(embeddings) + batch_size - 1) // batch_size

        for i in tqdm(range(0, len(embeddings), batch_size), total=total_batches, desc="Uploading"):
            batch_end = min(i + batch_size, len(embeddings))

            points = []
            for idx in range(i, batch_end):
                points.append(PointStruct(
                    id=start_id + idx,
                    vector=embeddings[idx].tolist(),
                    payload=chunks[idx]
                ))

            self.client.upsert(
                collection_name=self.collection_name,
                points=points
            )

        # Verify
        final_count = self.client.get_collection(self.collection_name).points_count
        print(f"\n✅ Upload complete!")
        print(f"📊 Total vectors in Qdrant: {final_count:,}")

    def run_auto_pipeline(self, drive_folder_path, mode="refresh"):
        """Complete auto pipeline: Auto-find CSVs → Qdrant"""
        print("="*60)
        print("🚀 QDRANT AUTO-UPDATE PIPELINE")
        print("="*60)

        # Step 1: Auto-find all CSV files
        csv_files = self.find_all_csv_files(drive_folder_path)
        if not csv_files:
            print("❌ No CSV files found. Exiting.")
            return

        # Step 2: Load and filter CSVs
        df_clean = self.load_and_filter_csvs(csv_files)
        if df_clean is None or len(df_clean) == 0:
            print("❌ No data to process. Exiting.")
            return

        # Step 3: Create chunks
        chunks = self.create_chunks(df_clean)
        if not chunks:
            print("❌ No chunks created. Exiting.")
            return

        # Step 4: Generate embeddings
        embeddings = self.generate_embeddings(chunks)

        # Step 5: Upload to Qdrant
        self.upload_to_qdrant(embeddings, chunks, mode=mode)

        print("\n" + "="*60)
        print("✅ PIPELINE COMPLETE!")
        print("="*60)
        print(f"📊 Your app now has access to {len(chunks):,} clinical trials")
        print("🔄 No code changes needed - just reload your app!")


# Usage
if __name__ == "__main__":
    import getpass

    # Configuration
    DRIVE_FOLDER = "/content/drive/MyDrive/LLM_Based_GenAI_Sem1/data"
    QDRANT_URL = "https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io"

    print("🔐 Qdrant Configuration")
    qdrant_key = getpass.getpass("Enter Qdrant API Key: ")

    print("\n📋 Update Mode:")
    print("1. refresh - Delete all old data and upload fresh")
    print("2. add - Keep existing data and add new data")
    mode_choice = input("Choose mode (1 or 2): ").strip()
    mode = "refresh" if mode_choice == "1" else "add"

    # Run pipeline
    pipeline = QdrantAutoPipeline(QDRANT_URL, qdrant_key)
    pipeline.run_auto_pipeline(DRIVE_FOLDER, mode=mode)


Overwriting update_qdrant_auto.py


In [ ]:
!python update_qdrant_auto.py

2025-11-30 16:26:56.042066: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764520016.065483   41049 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764520016.075744   41049 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764520016.097559   41049 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764520016.097585   41049 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764520016.097591   41049 computation_placer.cc:177] computation placer alr

In [ ]:
import pandas as pd

BASE = "/content/drive/MyDrive/LLM_Based_GenAI_Sem1/data"

df1 = pd.read_csv(f"{BASE}/clinical_trials_diabetes_full.csv")
df2 = pd.read_csv(f"{BASE}/clinical_trials_cancer_full.csv")
df3 = pd.read_csv(f"{BASE}/clinical_trials_alzheimer_full.csv")
df4 = pd.read_csv(f"{BASE}/clinical_trials_asthma_full.csv")
df5 = pd.read_csv(f"{BASE}/clinical_trials_cardiovascular_full.csv")
df6 = pd.read_csv(f"{BASE}/clinical_trials_master_full.csv")

# Check for duplicates
df_all = pd.concat([df1, df2, df3, df4, df5, df6], ignore_index=True)

print(f"Total rows: {len(df_all):,}")
print(f"Unique NCT IDs: {df_all['nct_id'].nunique():,}")
print(f"Duplicates: {len(df_all) - df_all['nct_id'].nunique():,}")

# Show example duplicates
duplicates = df_all[df_all.duplicated(subset=['nct_id'], keep=False)]
if len(duplicates) > 0:
    print(f"\n📋 Example duplicate NCT IDs:")
    print(duplicates.groupby('nct_id')['brief_title'].first().head())


Total rows: 419,386
Unique NCT IDs: 198,752
Duplicates: 220,634

📋 Example duplicate NCT IDs:
nct_id
NCT00000105    Vaccination With Tetanus and KLH to Assess Imm...
NCT00000107    Body Water Content in Cyanotic Congenital Hear...
NCT00000108    Effects of Training Intensity on the CHD Risk ...
NCT00000112    Prevalence of Carbohydrate Intolerance in Lean...
NCT00000124           Collaborative Ocular Melanoma Study (COMS)
Name: brief_title, dtype: object


Update Code to Use Qdrant Instead of FAISS

In [2]:
%%writefile utils_qdrant.py
import json
import hashlib
from datetime import datetime
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# --- Confidence score from distance ---

def calculate_confidence_score(distance: float, normalization_factor: float = 1.0) -> float:
    """Inverse L2 distance score in (0,1]; closer = higher confidence."""
    return normalization_factor / (normalization_factor + float(distance))


# --- Load Qdrant client + embedding model ---

def load_qdrant_and_model(qdrant_url: str, qdrant_api_key: str):
    """Loads Qdrant client and embedding model."""
    print("⏳ Connecting to Qdrant...")

    client = QdrantClient(url=qdrant_url, api_key=qdrant_api_key)

    # Verify connection
    collection_info = client.get_collection("clinical_trials")
    print(f"✅ Connected to Qdrant: {collection_info.points_count:,} vectors ready")

    # Load embedding model (same as before)
    embed_model = SentenceTransformer("all-MiniLM-L6-v2")
    print("✅ Embedding model loaded")

    return client, embed_model


# --- Provenance logging (unchanged) ---

def log_provenance_step(agent_name: str, input_data, output_data, detail=None):
    """Creates a detailed log entry for a single agent step."""
    log_entry = {
        "timestamp": datetime.now().isoformat(),
        "agent": agent_name,
        "input": input_data,
        "output": output_data,
        "detail": detail or {},
        "model_version": "gemini-2.0-flash",
    }
    return log_entry


# --- Reproducibility hash (unchanged) ---

def generate_reproducibility_hash(conversation_history, corpus_version: str = "v1.0"):
    """Generates a deterministic session hash based on conversation history."""
    queries = [turn.get("query", "") for turn in conversation_history]
    raw = f"{corpus_version}|{'|'.join(queries)}"
    return hashlib.md5(raw.encode("utf-8")).hexdigest()


Overwriting utils_qdrant.py


In [3]:
import getpass
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Get API key
qdrant_api_key = getpass.getpass("🔑 Qdrant API Key: ")

# Connect
qdrant_client = QdrantClient(
    url="https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io",
    api_key=qdrant_api_key
)

# Check what methods are available
print("Available search methods:")
print([m for m in dir(qdrant_client) if 'search' in m.lower() or 'query' in m.lower()])


🔑 Qdrant API Key: ··········
Available search methods:
['_resolve_query', '_resolve_query_batch_request', '_resolve_query_request', '_scored_points_to_query_responses', 'query', 'query_batch', 'query_batch_points', 'query_points', 'query_points_groups', 'search_matrix_offsets', 'search_matrix_pairs']


In [4]:
import getpass
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Get API key
qdrant_api_key = getpass.getpass("🔑 Qdrant API Key: ")

# Connect
qdrant_client = QdrantClient(
    url="https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io",
    api_key=qdrant_api_key
)

# Load model
embed_model = SentenceTransformer("all-MiniLM-L6-v2")

# Test search
test_query = "diabetes insulin therapy trials"
q_emb = embed_model.encode([test_query])[0]

# Use query_points (correct method)
results = qdrant_client.query_points(
    collection_name="clinical_trials",
    query=q_emb.tolist(),
    limit=3
)

print(f"\n🔍 Test Query: '{test_query}'")
print(f"\n📊 Top 3 Results:\n")

for i, point in enumerate(results.points, 1):
    print(f"{i}. NCT ID: {point.payload['nct_id']}")
    print(f"   Score: {point.score:.3f}")
    print(f"   Title: {point.payload['title'][:80]}...")
    print()

print("✅ Qdrant search working!")


🔑 Qdrant API Key: ··········


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



🔍 Test Query: 'diabetes insulin therapy trials'

📊 Top 3 Results:

1. NCT ID: NCT00151697
   Score: 0.752
   Title: LANN-study: Lantus, Amaryl, Novorapid, Novomix Study...

2. NCT ID: NCT02192424
   Score: 0.697
   Title: Early Intermittent Intensive Insulin Therapy as an Effective Treatment of Type 2...

3. NCT ID: NCT02846233
   Score: 0.697
   Title: Stepping-down Approach in Patients With Chronic Poorly-controlled Diabetes on Ad...

✅ Qdrant search working!


Fix RetrievalAgent to Use query_points

In [5]:
%%writefile retrieval_agent_qdrant.py
import numpy as np
from typing import Dict, Any
from qdrant_client import QdrantClient
from sentence_transformers import SentenceTransformer

# Import scoring from your existing code
from utils_qdrant import calculate_confidence_score, log_provenance_step

# Try to import reranker
CrossEncoder = None
try:
    from sentence_transformers import CrossEncoder
except ImportError:
    pass


class RetrievalAgentQdrant:
    """Retrieval agent using Qdrant instead of FAISS."""

    def __init__(
        self,
        qdrant_client: QdrantClient,
        embed_model: SentenceTransformer,
        evidence_scorer,
        profile_agent=None
    ):
        self.client = qdrant_client
        self.embed_model = embed_model
        self.evidence_scorer = evidence_scorer
        self.profile_agent = profile_agent

        # Optional: Load reranker
        self.reranker = None
        if CrossEncoder:
            try:
                print("⏳ Loading Cross-Encoder reranker...")
                self.reranker = CrossEncoder("cross-encoder/ms-marco-MiniLM-L-6-v2")
                print("✅ Reranker loaded")
            except Exception as e:
                print(f"⚠️ Reranker failed to load: {e}")

    def retrieve(self, parsed: Dict[str, Any], top_k: int = 5):
        """Retrieve trials from Qdrant."""
        FETCH_K = top_k * 3

        symptoms = parsed.get("symptoms") or []
        context = parsed.get("context") or ""
        query = parsed.get("user_question") or (" ".join(symptoms) + " " + context).strip()

        if not query:
            retrieval = {"query": "", "trials": [], "avg_confidence": 0.0}
            log = log_provenance_step("RetrievalAgentQdrant", parsed, retrieval, {"reason": "empty_query"})
            return retrieval, log

        # Query expansion (same as before)
        EXPANSIONS = {
            "insulin": "insulin OR insulin therapy OR insulin treatment OR insulin pump",
            "medication": "medication OR drug OR pharmaceutical OR pharmacological OR treatment",
            "diet": "diet OR dietary OR nutrition OR nutritional OR eating",
            "exercise": "exercise OR physical activity OR fitness OR activity",
            "chemo": "chemotherapy OR antineoplastic OR oncology",
            "cancer": "cancer OR tumor OR tumour OR malignancy OR oncology",
            "alzheim": "alzheimer OR dementia OR cognitive decline OR memory loss",
        }

        query_lower = query.lower()
        for term, expansion in EXPANSIONS.items():
            if term in query_lower:
                query = f"{query} {expansion}"
                break

        # 1. Generate query embedding
        q_emb = self.embed_model.encode([query])[0]

        # 2. Search Qdrant (FIXED: use query_points)
        search_results = self.client.query_points(
            collection_name="clinical_trials",
            query=q_emb.tolist(),
            limit=FETCH_K
        )

        # 3. Convert to candidate format
        initial_candidates = []
        for point in search_results.points:
            initial_candidates.append({
                "nct_id": point.payload["nct_id"],
                "title": point.payload.get("title", ""),
                "text": point.payload["text"],
                "status": point.payload.get("status", "Unknown Status"),
                "qdrant_score": point.score,  # Cosine similarity (higher = better)
            })

        final_trials = []

        # 4. Optional CrossEncoder reranking
        if self.reranker and initial_candidates:
            pairs = [[query, cand["text"]] for cand in initial_candidates]
            scores = self.reranker.predict(pairs)

            for i, cand in enumerate(initial_candidates):
                cand["rerank_score"] = float(scores[i])

            initial_candidates.sort(key=lambda x: x["rerank_score"], reverse=True)
            top_hits = initial_candidates[:top_k]

            for rank, item in enumerate(top_hits):
                logit = item["rerank_score"]
                base_conf = 1 / (1 + np.exp(-logit))

                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )

                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["title"],
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": scoring_result["weighted_score"],
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "qdrant_evidence_weighted",
                })
        else:
            # Qdrant-only path (no reranking)
            top_hits = initial_candidates[:top_k]
            for rank, item in enumerate(top_hits):
                # Qdrant uses cosine similarity (0-1, higher = better)
                base_conf = item["qdrant_score"]

                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )

                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["title"],
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": scoring_result["weighted_score"],
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "qdrant_evidence_weighted",
                })

        # Sort by weighted score
        final_trials.sort(key=lambda x: x["weighted_score"], reverse=True)
        for i, trial in enumerate(final_trials):
            trial["rank"] = i + 1

        confs = [t["weighted_score"] for t in final_trials]
        avg_conf = float(np.mean(confs)) if confs else 0.0

        retrieval = {
            "query": query,
            "trials": final_trials,
            "avg_confidence": avg_conf,
        }

        detail = {
            "top_k": top_k,
            "avg_confidence": avg_conf,
            "num_trials": len(final_trials),
            "method": "qdrant_reranked" if self.reranker else "qdrant_only",
        }

        log = log_provenance_step("RetrievalAgentQdrant", parsed, retrieval, detail)
        return retrieval, log


Overwriting retrieval_agent_qdrant.py


Update Main Bot Code to Use Qdrant

In [6]:
%%writefile run_bot_qdrant.py
"""
Updated HealthcareBot using Qdrant instead of FAISS
"""

import json
import re
import os
import sys
from typing import List, Dict, Any
import numpy as np
import requests
import google.generativeai as genai
from google.generativeai.types import HarmCategory, HarmBlockThreshold

# Import utilities
from utils_qdrant import (
    load_qdrant_and_model,
    log_provenance_step,
    generate_reproducibility_hash,
)

from retrieval_agent_qdrant import RetrievalAgentQdrant

# CrossEncoder
CrossEncoder = None
try:
    from sentence_transformers import CrossEncoder
except:
    pass


# ============================================================
# PARSER
# ============================================================
class SymptomParser:
    def __init__(self, model):
        self.model = model

    def parse(self, text: str):
        """
        Enhanced parser for clinical trial search queries.
        Decides:
        - Are they searching for trials or just asking a question?
        - Which disease (diabetes, cancer, Alzheimer’s, asthma, cardiovascular) is implied?
        """
        prompt = (
            "You are a clinical trial search classifier for medical research.\n"
            "You support conditions including diabetes, cancer, Alzheimer's disease, asthma, and cardiovascular disease.\n\n"
            f"User Input: \"{text}\"\n\n"
            "Your tasks:\n"
            "1) Decide if the user is searching for clinical trials or just asking a general question.\n"
            "2) Detect which disease(s) they are talking about.\n"
            "3) Detect if the query is not about health or clinical trials (off_topic).\n\n"
            "Classification Rules:\n"
            "- If the query mentions or implies trials, studies, research, clinical experiments, etc. → intent='trial_search'\n"
            "- If the user is mainly describing themselves (age, diagnosis, comorbidities, meds) → intent='profile_info'\n"
            "- If they ask 'what is X', 'how does Y work', etc. without asking about trials → intent='general_question'\n"
            "- Simple greetings (hi, hello, hey) → intent='greeting'\n"
            "- If clearly not about health or clinical research → intent='off_topic', is_disease_related=false\n\n"
            "You must detect disease_focus whenever possible:\n"
            "- diabetes: diabetes, blood sugar, glucose, insulin, HbA1c, metformin, GLP-1, SGLT2\n"
            "- cancer: cancer, tumor/tumour, chemotherapy, oncology, breast cancer, lung cancer, leukemia, lymphoma\n"
            "- alzheimers: Alzheimer's, dementia, memory loss, cognitive decline\n"
            "- asthma: asthma, wheezing, bronchodilator, inhaler\n"
            "- cardiovascular: heart failure, cardiovascular disease, hypertension, high blood pressure, angina,\n"
            "  myocardial infarction, coronary artery disease, stroke\n\n"
            "Return ONLY valid JSON with this exact format:\n"
            "{\n"
            "  \"intent\": \"trial_search\" | \"profile_info\" | \"general_question\" | \"greeting\" | \"off_topic\",\n"
            "  \"query_type\": \"trial_query\" | \"profile_statement\" | \"knowledge_seeking\" | \"greeting\",\n"
            "  \"search_keywords\": [\"keyword1\", \"keyword2\"],\n"
            "  \"is_disease_related\": true or false,\n"
            "  \"disease_focus\": [\"diabetes\", \"cancer\", \"alzheimers\", \"asthma\", \"cardiovascular\"],\n"
            "  \"user_question\": \"the question in plain English\",\n"
            "  \"trial_interest\": \"what type of trial they want (diet, medication, technology, surgery, etc.)\"\n"
            "}\n\n"
            "Examples:\n"
            "- 'What trials study liraglutide in diabetes?' → intent='trial_search', query_type='trial_query',\n"
            "  is_disease_related=true, disease_focus=['diabetes'], search_keywords=['liraglutide']\n"
            "- 'My mom has breast cancer, are there trials?' → intent='trial_search', disease_focus=['cancer']\n"
            "- 'I am 70 with memory loss and Alzheimer's' → intent='profile_info', disease_focus=['alzheimers']\n"
            "- 'What is HbA1c?' → intent='general_question', disease_focus=['diabetes']\n"
            "- 'What is the weather in Paris?' → intent='off_topic', is_disease_related=false, disease_focus=[]\n"
        )

        try:
            res = self.model.generate_content(prompt)
            raw = (res.text or "").strip()
            match = re.search(r"\{.*\}", raw, re.DOTALL)
            if match:
                parsed = json.loads(match.group(0))
            else:
                parsed = json.loads(raw)
        except Exception:
            # Fallback: simple heuristic if model fails
            text_lower = text.lower()
            disease_focus = []
            if any(x in text_lower for x in ["diabetes", "insulin", "glucose", "hba1c", "metformin", "glp-1", "sglt2"]):
                disease_focus.append("diabetes")
            if any(x in text_lower for x in ["cancer", "tumor", "tumour", "chemo", "chemotherapy", "oncology"]):
                disease_focus.append("cancer")
            if any(x in text_lower for x in ["alzheimer", "dementia", "memory loss", "cognitive decline"]):
                disease_focus.append("alzheimers")
            if "asthma" in text_lower or "wheezing" in text_lower:
                disease_focus.append("asthma")
            if any(x in text_lower for x in ["heart failure", "cardiovascular", "hypertension",
                                             "high blood pressure", "angina", "myocardial", "coronary", "stroke"]):
                disease_focus.append("cardiovascular")

            if any(kw in text_lower for kw in ["trial", "study", "research", "clinical"]):
                intent = "trial_search"
                query_type = "trial_query"
            elif any(kw in text_lower for kw in ["hi", "hello", "hey"]):
                intent = "greeting"
                query_type = "greeting"
            else:
                intent = "general_question"
                query_type = "knowledge_seeking"

            parsed = {
                "intent": intent,
                "query_type": query_type,
                "search_keywords": [text] if intent == "trial_search" else [],
                "is_disease_related": bool(disease_focus),
                "disease_focus": disease_focus,
                "user_question": text,
                "trial_interest": "general",
            }

        # --- Heuristic correction layer on top of model output ---
        text_lower = text.lower()
        diseases = set(parsed.get("disease_focus") or [])

        if any(x in text_lower for x in ["diabetes", "insulin", "glucose", "hba1c", "metformin", "glp-1", "sglt2"]):
            diseases.add("diabetes")
        if any(x in text_lower for x in ["cancer", "tumor", "tumour", "chemo", "chemotherapy", "oncology"]):
            diseases.add("cancer")
        if any(x in text_lower for x in ["alzheimer", "alzheimers", "dementia", "memory loss", "cognitive decline"]):
            diseases.add("alzheimers")
        if "asthma" in text_lower or "wheezing" in text_lower or "inhaler" in text_lower:
            diseases.add("asthma")
        if any(x in text_lower for x in ["heart failure", "cardiovascular", "hypertension",
                                         "high blood pressure", "angina", "myocardial", "coronary", "stroke"]):
            diseases.add("cardiovascular")

        parsed["disease_focus"] = list(diseases)

        # Force trial_search if obvious trial keywords
        trial_keywords = [
            "trial", "study", "studies", "research",
            "clinical", "show me", "are there", "what trials"
        ]
        if any(kw in text_lower for kw in trial_keywords):
            parsed["intent"] = "trial_search"
            parsed["query_type"] = "trial_query"

        # If we detected diseases, ensure is_disease_related = True
        if diseases and parsed.get("intent") != "off_topic":
            parsed["is_disease_related"] = True
        elif "is_disease_related" not in parsed:
            parsed["is_disease_related"] = bool(diseases)

        log = log_provenance_step("SymptomParser", text, parsed)
        return parsed, log


# ============================================================
# PROFILE AGENT
# ============================================================
class ProfileAgent:
    def __init__(self, initial_profile: Dict[str, Any] = None):
        if initial_profile is None:
            initial_profile = {
                "user_id": "Patient",
                "conditions": [],          # could be filled later
                "extracted_conditions": [],  # dynamic memory
                "history": [],
            }
        self.profile = initial_profile

    def update_profile(self, turn_data: Dict[str, Any]):
        """
        Updates history and extracts persistent medical entities.
        """
        self.profile.setdefault("history", []).append(turn_data)
        self.profile.setdefault("extracted_conditions", [])

        parsed = turn_data.get("parsed", {})
        # optional: keep disease_focus as conditions
        diseases = parsed.get("disease_focus") or []
        if diseases:
            current = set(self.profile["extracted_conditions"])
            for d in diseases:
                current.add(d)
            self.profile["extracted_conditions"] = list(current)

        snapshot = {
            "user_id": self.profile.get("user_id", "Patient"),
            "known_conditions": self.profile.get("extracted_conditions", []),
            "num_turns": len(self.profile["history"]),
        }
        log = log_provenance_step("ProfileAgent", turn_data, {"profile_snapshot": snapshot})
        return log


# ============================================================
# EVIDENCE-WEIGHTED SCORER
# ============================================================
class EvidenceWeightedScorer:
    """
    Implements evidence-weighted scoring for clinical trials.
    Ranks trials based on multiple quality factors beyond semantic similarity.
    """

    def __init__(self):
        self.status_weights = {
            "Completed": 1.0,
            "Active, Not Recruiting": 0.9,
            "Recruiting": 0.85,
            "Enrolling By Invitation": 0.8,
            "Not Yet Recruiting": 0.6,
            "Terminated": 0.4,
            "Withdrawn": 0.3,
            "Suspended": 0.3,
            "Unknown Status": 0.5,
        }

        self.design_keywords = {
            "randomized controlled": 1.0,
            "double-blind": 0.95,
            "randomized": 0.9,
            "controlled": 0.85,
            "interventional": 0.8,
            "prospective": 0.75,
            "observational": 0.6,
            "retrospective": 0.5,
        }

    def calculate_weighted_score(
        self,
        trial: Dict[str, Any],
        base_confidence: float,
        query: str,
    ) -> Dict[str, Any]:
        """
        Calculate evidence-weighted score for a trial.
        """

        # Factor 1: Base semantic match (35%)
        match_score = base_confidence * 0.35

        # Factor 2: Trial status quality (25%)
        status = str(trial.get("status", "Unknown Status")).strip().title()
        status_score = self.status_weights.get(status, 0.5) * 0.25

        # Factor 3: Study design quality (20%)
        design_score = self._calculate_design_quality(trial) * 0.20

        # Factor 4: Keyword density (10%)
        keyword_score = self._calculate_keyword_density(trial, query) * 0.10

        # Factor 5: Metadata completeness (10%)
        completeness_score = self._calculate_completeness(trial) * 0.10

        weighted_score = (
            match_score +
            status_score +
            design_score +
            keyword_score +
            completeness_score
        )

        breakdown = {
            "base_confidence": base_confidence,
            "weighted_score": weighted_score,
            "factors": {
                "semantic_match": match_score,
                "trial_status": status_score,
                "study_design": design_score,
                "keyword_density": keyword_score,
                "completeness": completeness_score,
            },
        }

        return {
            "weighted_score": min(weighted_score, 1.0),
            "breakdown": breakdown,
        }

    def _calculate_design_quality(self, trial: Dict[str, Any]) -> float:
        text = f"{trial.get('title', '')} {trial.get('text', '')}".lower()
        max_score = 0.0
        for keyword, weight in self.design_keywords.items():
            if keyword in text:
                max_score = max(max_score, weight)
        return max_score if max_score > 0 else 0.6

    def _calculate_keyword_density(self, trial: Dict[str, Any], query: str) -> float:
        text = f"{trial.get('title', '')} {trial.get('text', '')}".lower()
        stopwords = {
            "the", "a", "an", "and", "or", "for", "with", "in", "on", "at", "to",
            "of", "is", "are", "what", "trials", "trial", "study", "studies", "clinical"
        }
        query_terms = [
            term for term in query.lower().split()
            if term not in stopwords and len(term) > 2
        ]
        if not query_terms:
            return 0.5
        matches = sum(1 for term in query_terms if term in text)
        density = matches / len(query_terms)
        return min(density, 1.0)

    def _calculate_completeness(self, trial: Dict[str, Any]) -> float:
        # Our chunk_map has "title" and "text"; treat longer text as more complete
        text = trial.get("text", "") or ""
        title = trial.get("title", "") or ""
        score = 0.0
        if len(title) > 10:
            score += 0.3
        if len(text) > 200:
            score += 0.7
        return min(score, 1.0)


# ============================================================
# PubMed Helper (NCT → PubMed abstract)
# ============================================================
def fetch_pubmed_abstract_for_nct(nct_id: str):
    """
    Try to find a PubMed article linked to this NCT ID and return its abstract.
    Returns: {"pmid": str, "abstract": str} or None
    """
    try:
        esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
        params = {
            "db": "pubmed",
            "term": f"{nct_id}[si]",
            "retmode": "json",
            "retmax": 1,
        }
        r = requests.get(esearch_url, params=params, timeout=10)
        r.raise_for_status()
        data = r.json()
        idlist = data.get("esearchresult", {}).get("idlist", [])
        if not idlist:
            return None

        pmid = idlist[0]

        efetch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi"
        params = {
            "db": "pubmed",
            "id": pmid,
            "rettype": "abstract",
            "retmode": "text",
        }
        r2 = requests.get(efetch_url, params=params, timeout=10)
        r2.raise_for_status()
        abstract_text = r2.text.strip()
        if not abstract_text:
            return None

        return {"pmid": pmid, "abstract": abstract_text}
    except Exception:
        return None


# ============================================================
# RETRIEVAL AGENT
# ============================================================
class RetrievalAgent:
    def __init__(self, embed_model, faiss_index, chunk_map, profile_agent: ProfileAgent = None):
        self.embed_model = embed_model
        self.index = faiss_index
        self.chunk_map = chunk_map
        self.profile_agent = profile_agent
        self.evidence_scorer = EvidenceWeightedScorer()

    def retrieve(self, parsed: Dict[str, Any], top_k: int = 5):
        FETCH_K = top_k * 3

        symptoms = parsed.get("symptoms") or []
        context = parsed.get("context") or ""
        query = parsed.get("user_question") or (" ".join(symptoms) + " " + context).strip()

        if not query:
            retrieval = {"query": "", "trials": [], "avg_confidence": 0.0}
            log = log_provenance_step("RetrievalAgent", parsed, retrieval, {"reason": "empty_query"})
            return retrieval, log

        # Simple expansions (still useful across diseases)
        EXPANSIONS = {
            "insulin": "insulin OR insulin therapy OR insulin treatment OR insulin pump",
            "medication": "medication OR drug OR pharmaceutical OR pharmacological OR treatment",
            "diet": "diet OR dietary OR nutrition OR nutritional OR eating",
            "exercise": "exercise OR physical activity OR fitness OR activity",
            "chemo": "chemotherapy OR antineoplastic OR oncology",
            "cancer": "cancer OR tumor OR tumour OR malignancy OR oncology",
            "alzheim": "alzheimer OR dementia OR cognitive decline OR memory loss",
        }

        query_lower = query.lower()
        for term, expansion in EXPANSIONS.items():
            if term in query_lower:
                query = f"{query} {expansion}"
                break

        # 1. FAISS retrieval
        q_emb = self.embed_model.encode([query])
        distances, indices = self.index.search(q_emb.astype("float32"), FETCH_K)

        initial_candidates = []
        for rank, idx in enumerate(indices[0]):
            if idx == -1:
                continue
            item = self.chunk_map[idx]
            dist = float(distances[0][rank])
            initial_candidates.append({
                "nct_id": item["nct_id"],
                "title": item.get("title", ""),
                "text": item["text"],
                "status": item.get("status", "Unknown Status"),
                "faiss_dist": dist,
            })

        final_trials = []
        confs = []

        # 2. Optional CrossEncoder reranking
        if reranker and initial_candidates:
            pairs = [[query, cand["text"]] for cand in initial_candidates]
            scores = reranker.predict(pairs)

            for i, cand in enumerate(initial_candidates):
                cand["rerank_score"] = float(scores[i])

            initial_candidates.sort(key=lambda x: x["rerank_score"], reverse=True)
            top_hits = initial_candidates[:top_k]

            for rank, item in enumerate(top_hits):
                logit = item["rerank_score"]
                base_conf = 1 / (1 + np.exp(-logit))

                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )

                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["title"],
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": scoring_result["weighted_score"],
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "evidence_weighted",
                })

            final_trials.sort(key=lambda x: x["weighted_score"], reverse=True)
            for i, trial in enumerate(final_trials):
                trial["rank"] = i + 1

            confs = [t["weighted_score"] for t in final_trials]

        else:
            # FAISS-only path
            top_hits = initial_candidates[:top_k]
            for rank, item in enumerate(top_hits):
                base_conf = calculate_confidence_score(item["faiss_dist"])
                scoring_result = self.evidence_scorer.calculate_weighted_score(
                    trial=item,
                    base_confidence=base_conf,
                    query=query,
                )
                final_trials.append({
                    "nct_id": item["nct_id"],
                    "title": item["title"],
                    "text": item["text"],
                    "status": item["status"],
                    "confidence": base_conf,
                    "weighted_score": scoring_result["weighted_score"],
                    "score_breakdown": scoring_result["breakdown"],
                    "rank": rank + 1,
                    "method": "evidence_weighted_faiss",
                })

            final_trials.sort(key=lambda x: x["weighted_score"], reverse=True)
            for i, trial in enumerate(final_trials):
                trial["rank"] = i + 1

            confs = [t["weighted_score"] for t in final_trials]

        avg_conf = float(np.mean(confs)) if confs else 0.0

        retrieval = {
            "query": query,
            "trials": final_trials,
            "avg_confidence": avg_conf,
        }

        detail = {
            "top_k": top_k,
            "avg_confidence": avg_conf,
            "num_trials": len(final_trials),
            "method": "reranked" if reranker else "faiss_only",
        }

        log = log_provenance_step("RetrievalAgent", parsed, retrieval, detail)
        return retrieval, log


# ============================================================
# DIAGNOSIS / ADVISOR
# ============================================================
class DiagnosisAdvisor:
    def __init__(self, model):
        self.model = model

    def _handle_general_question(self, parsed: Dict[str, Any], retrieved: Dict[str, Any]):
        """Handle general medical knowledge questions."""
        trials = retrieved.get("trials", [])
        user_question = parsed.get("user_question") or " ".join(parsed.get("symptoms", []))

        evidence_parts = []
        for t in trials[:3]:
            evidence_parts.append(f"Trial {t['nct_id']}: {t['text'][:400]}")
        evidence = "\n\n".join(evidence_parts) if evidence_parts else "No specific trials available."

        prompt = (
            "You are a medical research educator. Answer the user's question clearly using reliable medical knowledge.\n"
            "The clinical trial evidence below provides real-world context - mention it if helpful.\n\n"
            f"USER'S QUESTION: {user_question}\n\n"
            "CLINICAL TRIAL CONTEXT (for reference only):\n"
            f"{evidence}\n\n"
            "Instructions:\n"
            "- Answer the question directly in 3–5 sentences.\n"
            "- Be specific and educational.\n"
            "- Do NOT give diagnoses or treatment instructions.\n"
            "- End with: 'For personalized advice, please consult your healthcare provider.'\n"
        )

        try:
            res = self.model.generate_content(prompt)
            text = (res.text or "").strip()
            if not text or len(text) < 50:
                text = (
                    "I don't have enough information to answer this question accurately. "
                    "For personalized guidance, please consult your healthcare provider."
                )
            return text
        except Exception:
            return (
                "I'm unable to generate a detailed answer right now. "
                "For personalized guidance, please consult your healthcare provider."
            )

    def _handle_symptom_query(
        self,
        parsed: Dict[str, Any],
        retrieved: Dict[str, Any],
        profile: Dict[str, Any],
    ):
        """
        Generate response for clinical trial search queries with
        readable paragraph summaries and PubMed abstracts when available.
        """
        trials = retrieved.get("trials", [])
        if not trials:
            return "No relevant trials were found. Please try refining your query."

        formatted_trials = []
        for t in trials[:5]:
            title = t.get("title", "") or t["text"].split("\n")[0].replace("Title: ", "")
            status = t.get("status", "Unknown")
            weighted_score = t.get("weighted_score", 0.0)

            # Extract the ClinicalTrials.gov summary text
            raw_text = t.get("text", "")
            brief_summary = raw_text.split("Summary:", 1)[-1].strip() if "Summary:" in raw_text else raw_text

            if brief_summary:
                # Ask Gemini to turn the CT.gov summary into a short paragraph
                prompt = (
                    "Rewrite the following clinical trial description as a short, clear paragraph "
                    "about what the study is testing:\n\n"
                    f"{brief_summary}\n\n"
                    "Guidelines:\n"
                    "- Use 2–4 sentences.\n"
                    "- Plain English, minimal jargon.\n"
                    "- Include the purpose and the main type of participant.\n"
                )
                try:
                    res = self.model.generate_content(prompt)
                    brief_summary = res.text.strip() if res.text else brief_summary
                except Exception:
                    if len(brief_summary) > 600:
                        brief_summary = brief_summary[:600] + "..."
            else:
                brief_summary = "No summary available."

            # PubMed abstract lookup
            pubmed_block = ""
            pub = fetch_pubmed_abstract_for_nct(t["nct_id"])
            if pub:
                abs_text = pub["abstract"]
                max_len = 2000
                if len(abs_text) > max_len:
                    abs_text = abs_text[:max_len] + "..."
                pubmed_block = (
                    f"  PubMed abstract (PMID {pub['pmid']}):\n"
                    f"  {abs_text}\n\n"
                    f"  PubMed link: https://pubmed.ncbi.nlm.nih.gov/{pub['pmid']}/\n\n"
                )

            formatted_trials.append(
                f"**{t['nct_id']}** (Relevance: {weighted_score:.0%})\n"
                f"• {title}\n"
                f"  Status: {status}\n\n"
                f"  {brief_summary}\n\n"
                f"{pubmed_block}"
            )

        trials_text = "\n\n".join(formatted_trials)
        num_trials = len(formatted_trials)

        response = (
            f"I found {num_trials} clinical trial{'s' if num_trials != 1 else ''} relevant to your request:\n\n"
            f"{trials_text}\n\n"
            "Summary: These trials explore potential treatments or management strategies for the condition you asked about. "
            "More details are available using the listed NCT IDs.\n\n"
            "To learn more or consider participation, visit clinicaltrials.gov and search by NCT ID. "
            "Always discuss clinical trial options with your healthcare provider."
        )

        return response

    def advise(self, parsed: Dict[str, Any], retrieved: Dict[str, Any], profile: Dict[str, Any]):
        trials = retrieved.get("trials", [])
        avg_conf = retrieved.get("avg_confidence", 0.0)
        query_type = parsed.get("query_type", "trial_query")
        is_disease_related = parsed.get("is_disease_related", True)

        draft = {
            "recommendation": "",
            "avg_confidence": avg_conf,
            "query_type": query_type,
        }

        if not is_disease_related:
            draft["recommendation"] = (
                "I’m specialized in clinical trials for medical conditions (for example diabetes, cancer, "
                "Alzheimer’s disease, asthma, and cardiovascular diseases). "
                "Your question does not appear to be about a health condition or clinical research. "
                "If you’d like, you can ask me about trials for a specific condition."
            )
            draft["confidence_veto"] = True
            log = log_provenance_step(
                "DiagnosisAdvisor",
                parsed,
                draft,
                {"veto": True, "reason": "off_topic"},
            )
            return draft, log

        if not trials or avg_conf < 0.05:
            draft["recommendation"] = (
                "Based on the trials I retrieved, I don’t have strong enough evidence to answer this question directly. "
                "Please consult your healthcare provider for personalized advice."
            )
            draft["confidence_veto"] = True
            log = log_provenance_step(
                "DiagnosisAdvisor",
                parsed,
                draft,
                {"veto": True, "reason": "low_confidence"},
            )
            return draft, log

        if query_type == "knowledge_seeking":
            draft["recommendation"] = self._handle_general_question(parsed, retrieved)
        else:
            draft["recommendation"] = self._handle_symptom_query(parsed, retrieved, profile)

        draft["confidence_veto"] = False
        log = log_provenance_step("DiagnosisAdvisor", parsed, draft)
        return draft, log


# ============================================================
# SAFETY FILTER
# ============================================================
class ActiveSafetyFilter:
    def __init__(self, model):
        self.model = model
        self.safety_cfg = {
            HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT: HarmBlockThreshold.BLOCK_NONE,
        }

    def verify(self, advice_text: str, trials: List[Dict[str, Any]]):
        # Skip safety check for list-type responses about trials
        if any(marker in advice_text for marker in ["NCT", "clinical trial", "clinicaltrials.gov"]):
            log = log_provenance_step(
                "ActiveSafetyFilter",
                {"advice": advice_text},
                {"final_text": advice_text, "status": "Pass (Trial Listing)"},
            )
            return advice_text, "Pass (Trial Listing)", log

        evidence_text = "\n".join(t["text"][:500] for t in trials[:3])

        audit_prompt = (
            "You are a Medical Safety Officer reviewing AI-generated advice.\n\n"
            "ADVICE:\n"
            f"{advice_text}\n\n"
            "EVIDENCE FROM CLINICAL TRIALS (for context):\n"
            f"{evidence_text}\n\n"
            "Check for safety issues:\n"
            "- If the advice suggests starting/stopping/changing medication without a doctor → UNSAFE.\n"
            "- If it gives a diagnosis → UNSAFE.\n"
            "- If it makes strong clinical claims not supported by evidence → UNSAFE.\n"
            "- If it simply lists clinical trials with neutral wording and a recommendation to talk to a doctor → SAFE.\n\n"
            "If the advice is acceptable, respond with exactly: SAFE\n"
            "If it is not acceptable, respond starting with: CORRECTED: <safer version>\n"
        )

        try:
            res = self.model.generate_content(audit_prompt, safety_settings=self.safety_cfg)
            txt = (res.text or "").strip()
            if txt.startswith("SAFE") or "SAFE" in txt:
                final_text = advice_text
                status = "Pass"
            else:
                final_text = f"⚠️ SAFETY REVISION:\n{txt}"
                status = "Revised"
        except Exception:
            if "NCT" in advice_text or "clinical trial" in advice_text.lower():
                final_text = advice_text
                status = "Pass (API Fallback)"
            else:
                final_text = "⚠️ Safety filter triggered. Please consult a doctor."
                status = "Revised (API Error)"

        log = log_provenance_step(
            "ActiveSafetyFilter",
            {"advice": advice_text},
            {"final_text": final_text, "status": status},
        )
        return final_text, status, log


# ============================================================
# HEALTHCAREBOT - Updated to use Qdrant
# ============================================================

class HealthcareBot:
    def __init__(self, qdrant_client, embed_model, gemini_model, initial_profile=None):
        self.parser = SymptomParser(gemini_model)
        self.profile_agent = ProfileAgent(initial_profile)
        self.evidence_scorer = EvidenceWeightedScorer()

        # NEW: Use Qdrant retrieval agent
        self.retrieval = RetrievalAgentQdrant(
            qdrant_client=qdrant_client,
            embed_model=embed_model,
            evidence_scorer=self.evidence_scorer,
            profile_agent=self.profile_agent
        )

        self.advisor = DiagnosisAdvisor(gemini_model)
        self.safety_filter = ActiveSafetyFilter(gemini_model)
        self.conversation_history = []
        self.provenance_log = []


    def chat(self, user_input: str) -> Dict[str, Any]:
        """Process user input through the pipeline."""

        # Parse intent
        parsed, parse_log = self.parser.parse(user_input)
        self.provenance_log.append(parse_log)

        # Update profile
        turn_data = {"query": user_input, "parsed": parsed}
        profile_log = self.profile_agent.update_profile(turn_data)
        self.provenance_log.append(profile_log)

        # Retrieve trials (now from Qdrant!)
        retrieved, retrieval_log = self.retrieval.retrieve(parsed, top_k=5)
        self.provenance_log.append(retrieval_log)

        # Generate response
        profile_snapshot = {
            "user_id": self.profile_agent.profile.get("user_id", "Patient"),
            "known_conditions": self.profile_agent.profile.get("extracted_conditions", []),
        }


        draft, advisor_log = self.advisor.advise(parsed, retrieved, profile_snapshot)
        self.provenance_log.append(advisor_log)

        # Safety filter - FIXED: verify() returns (text, status, log)
        advice_text = draft.get("recommendation", "") if isinstance(draft, dict) else str(draft)
        trials = retrieved.get("trials", [])

        final_response, safety_status, safety_log = self.safety_filter.verify(advice_text, trials)
        self.provenance_log.append(safety_log)

        # Save turn
        full_turn = {
            "query": user_input,
            "parsed": parsed,
            "retrieved": retrieved,
            "response": final_response,
            "timestamp": parse_log["timestamp"],
        }
        self.conversation_history.append(full_turn)

        return {
            "response": final_response,
            "avg_confidence": retrieved.get("avg_confidence", 0.0),
            "num_trials": len(retrieved.get("trials", [])),
            "provenance": self.provenance_log[-5:],
            "session_hash": generate_reproducibility_hash(self.conversation_history),
        }



def run_bot(user_input: str, qdrant_client, embed_model, gemini_model) -> Dict[str, Any]:
    """Convenience wrapper for single queries."""
    bot = HealthcareBot(qdrant_client, embed_model, gemini_model)
    return bot.chat(user_input)

Overwriting run_bot_qdrant.py


In [ ]:
# # Quick check - what does verify expect?
# import inspect
# from run_bot_qdrant import ActiveSafetyFilter

# print(inspect.signature(ActiveSafetyFilter.verify))


(self, advice_text: str, trials: List[Dict[str, Any]])


In [ ]:
# !sed -n '805,812p' /content/run_bot_qdrant.py


        # Safety filter - FIXED: verify() returns (text, status, log)
        advice_text = draft.get("recommendation", "") if isinstance(draft, dict) else str(draft)
        trials = retrieved.get("trials", [])

        final_response, safety_status, safety_log = self.safety_filter.verify(advice_text, trials)
        self.provenance_log.append(safety_log)

        # Save turn


In [ ]:
# # Force reload the module
# import sys
# if 'run_bot_qdrant' in sys.modules:
#     del sys.modules['run_bot_qdrant']

# # Now import fresh
# from run_bot_qdrant import run_bot

# # Test again
# result = run_bot(
#     "What trials are studying insulin therapy for diabetes?",
#     qdrant_client,
#     embed_model,
#     gemini_model
# )


⏳ Loading Cross-Encoder reranker...
✅ Reranker loaded


In [ ]:
# import inspect
# from run_bot_qdrant import ActiveSafetyFilter

# # Show the FULL source code of verify
# print(inspect.getsource(ActiveSafetyFilter.verify))


    def verify(self, advice_text: str, trials: List[Dict[str, Any]]):
        # Skip safety check for list-type responses about trials
        if any(marker in advice_text for marker in ["NCT", "clinical trial", "clinicaltrials.gov"]):
            log = log_provenance_step(
                "ActiveSafetyFilter",
                {"advice": advice_text},
                {"final_text": advice_text, "status": "Pass (Trial Listing)"},
            )
            return advice_text, "Pass (Trial Listing)", log

        evidence_text = "\n".join(t["text"][:500] for t in trials[:3])

        audit_prompt = (
            "You are a Medical Safety Officer reviewing AI-generated advice.\n\n"
            "ADVICE:\n"
            f"{advice_text}\n\n"
            "EVIDENCE FROM CLINICAL TRIALS (for context):\n"
            f"{evidence_text}\n\n"
            "Check for safety issues:\n"
            "- If the advice suggests starting/stopping/changing medication without a doctor → UNSAFE.\n"
            "

In [ ]:
# # Kill all cached modules
# import sys
# for key in list(sys.modules.keys()):
#     if any(x in key for x in ['run_bot', 'utils_qdrant', 'retrieval_agent']):
#         del sys.modules[key]

# print("✅ Modules cleared")


✅ Modules cleared


In [7]:
import os
import getpass
import google.generativeai as genai

# Setup keys
gemini_key = getpass.getpass("🔑 Gemini API Key: ")
os.environ["GEMINI_API_KEY"] = gemini_key

qdrant_api_key = getpass.getpass("🔑 Qdrant API Key: ")

# Setup clients
genai.configure(api_key=gemini_key)
gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

from utils_qdrant import load_qdrant_and_model
qdrant_url = "https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io"
qdrant_client, embed_model = load_qdrant_and_model(qdrant_url, qdrant_api_key)

# Import and test
from run_bot_qdrant import run_bot

print("\n🤖 Testing bot...\n")
result = run_bot(
    "What trials are studying insulin therapy for diabetes?",
    qdrant_client,
    embed_model,
    gemini_model
)

print(result["response"])
print(f"\n📊 Found {result['num_trials']} trials")


🔑 Gemini API Key: ··········
🔑 Qdrant API Key: ··········
⏳ Connecting to Qdrant...
✅ Connected to Qdrant: 146,593 vectors ready
✅ Embedding model loaded

🤖 Testing bot...

⏳ Loading Cross-Encoder reranker...
✅ Reranker loaded
I found 5 clinical trials relevant to your request:

**NCT04981808** (Relevance: 96%)
• Diabetes teleMonitoring of Patients in Insulin Therapy
  Status: Completed

  This study is testing whether monitoring patients with type 2 diabetes at home using electronic devices can improve their health. Participants who are already using insulin will be randomly assigned to either a group that uses home monitoring or a group that receives standard care. For three months, the health data from the home monitoring group will be tracked by hospital staff.

  PubMed abstract (PMID 36476605):
  1. Trials. 2022 Dec 7;23(1):985. doi: 10.1186/s13063-022-06921-6.

The Diabetes teleMonitoring of patients in insulin Therapy (DiaMonT) trial: 
study protocol for a randomized controlled

Update Streamlit App

In [8]:
%%writefile app.py
"""
Streamlit UI for HealthcareBot with Qdrant backend
"""

import streamlit as st
import os
from typing import Dict, Any
import google.generativeai as genai

# Import Qdrant utilities and bot
from utils_qdrant import load_qdrant_and_model
from run_bot_qdrant import HealthcareBot

# Page config
st.set_page_config(
    page_title="Clinical Trials Search Assistant",
    page_icon="🏥",
    layout="wide"
)

# Title
st.title("🏥 Clinical Trials Search Assistant")
st.markdown("**Powered by Qdrant + Gemini 2.0 Flash**")
st.markdown("Search across 262,000+ clinical trials for diabetes, cancer, Alzheimer's, asthma, and cardiovascular disease.")

# Sidebar for API keys
with st.sidebar:
    st.header("⚙️ Configuration")

    # Gemini API Key
    gemini_key = st.text_input(
        "Gemini API Key",
        type="password",
        help="Enter your Gemini API key"
    )

    # Qdrant API Key
    qdrant_key = st.text_input(
        "Qdrant API Key",
        type="password",
        help="Enter your Qdrant API key"
    )

    # Qdrant URL (pre-filled)
    qdrant_url = st.text_input(
        "Qdrant Cluster URL",
        value="https://215ec69e-fa22-4f38-bcf3-941e73901a68.us-east4-0.gcp.cloud.qdrant.io",
        help="Your Qdrant cluster URL"
    )

    st.divider()

    # Info
    st.markdown("### 📊 System Status")
    if gemini_key and qdrant_key:
        st.success("✅ Keys configured")
    else:
        st.warning("⚠️ Enter API keys to start")

# Initialize session state
if "messages" not in st.session_state:
    st.session_state.messages = []

if "bot" not in st.session_state:
    st.session_state.bot = None

if "qdrant_client" not in st.session_state:
    st.session_state.qdrant_client = None

if "embed_model" not in st.session_state:
    st.session_state.embed_model = None

# Initialize bot when keys are provided
if gemini_key and qdrant_key and st.session_state.bot is None:
    with st.spinner("🔄 Initializing system..."):
        try:
            # Setup Gemini
            os.environ["GEMINI_API_KEY"] = gemini_key
            genai.configure(api_key=gemini_key)
            gemini_model = genai.GenerativeModel("models/gemini-2.0-flash")

            # Setup Qdrant
            qdrant_client, embed_model = load_qdrant_and_model(qdrant_url, qdrant_key)

            # Store in session
            st.session_state.qdrant_client = qdrant_client
            st.session_state.embed_model = embed_model

            # Initialize bot
            st.session_state.bot = HealthcareBot(
                qdrant_client=qdrant_client,
                embed_model=embed_model,
                gemini_model=gemini_model
            )

            st.success("✅ System ready!")

        except Exception as e:
            st.error(f"❌ Initialization failed: {str(e)}")

# Display chat messages
for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

        # Show metadata for assistant messages
        if message["role"] == "assistant" and "metadata" in message:
            with st.expander("📊 Details"):
                col1, col2 = st.columns(2)
                with col1:
                    st.metric("Trials Found", message["metadata"]["num_trials"])
                with col2:
                    st.metric("Confidence", f"{message['metadata']['avg_confidence']:.0%}")

# Chat input
if prompt := st.chat_input("Ask about clinical trials..."):

    # Check if bot is initialized
    if st.session_state.bot is None:
        st.error("⚠️ Please enter your API keys in the sidebar first!")
    else:
        # Add user message
        st.session_state.messages.append({"role": "user", "content": prompt})
        with st.chat_message("user"):
            st.markdown(prompt)

        # Get bot response
        with st.chat_message("assistant"):
            with st.spinner("🔍 Searching clinical trials..."):
                try:
                    result = st.session_state.bot.chat(prompt)

                    response = result["response"]

                    # Display response
                    st.markdown(response)

                    # Show metadata
                    with st.expander("📊 Details"):
                        col1, col2, col3 = st.columns(3)
                        with col1:
                            st.metric("Trials Found", result["num_trials"])
                        with col2:
                            st.metric("Avg Confidence", f"{result['avg_confidence']:.0%}")
                        with col3:
                            st.metric("Session Hash", result["session_hash"][:8])

                    # Add to messages
                    st.session_state.messages.append({
                        "role": "assistant",
                        "content": response,
                        "metadata": {
                            "num_trials": result["num_trials"],
                            "avg_confidence": result["avg_confidence"]
                        }
                    })

                except Exception as e:
                    st.error(f"❌ Error: {str(e)}")
                    st.exception(e)

# Sidebar examples
with st.sidebar:
    st.divider()
    st.markdown("### 💡 Example Queries")

    examples = [
        "What trials study insulin therapy for diabetes?",
        "Show me cancer immunotherapy trials",
        "Are there trials for Alzheimer's disease?",
        "What trials are recruiting for asthma?",
        "Find cardiovascular disease trials"
    ]

    for example in examples:
        if st.button(example, key=example):
            st.session_state.messages.append({"role": "user", "content": example})
            st.rerun()

# Footer
st.divider()
st.markdown("""
<div style='text-align: center; color: gray; font-size: 0.9em;'>
    🔬 Powered by Qdrant Vector Database + Gemini 2.0 Flash<br>
    📊 Searching 262,660+ clinical trials across 5 disease areas
</div>
""", unsafe_allow_html=True)


Overwriting app.py


In [9]:
# Install Streamlit and pyngrok
!pip install -q streamlit pyngrok

print("✅ Packages installed!")


✅ Packages installed!


In [10]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64
!mv cloudflared-linux-amd64 cloudflared
!chmod +x cloudflared

In [11]:
#AI LLM
!streamlit run app.py &>/dev/null&
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-30T17:14:34Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-30T17:14:34Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-30T17:14:37Z INF +--------------------------------------------------------------------------------------------+
2025-11-30T17:14:37Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-30T17:14:37Z INF |  https://css-delhi-skilled-babies.trycloudflare.com   

In [12]:
1

1

In [13]:
from google.colab import files
import os

# Create a list of files to download
files_to_download = [
    'app.py',
    'run_bot_qdrant.py',
    'utils_qdrant.py',
    'retrieval_agent_qdrant.py'
]

# Download each file
for file in files_to_download:
    if os.path.exists(f'/content/{file}'):
        print(f"📥 Downloading {file}...")
        files.download(f'/content/{file}')
    else:
        print(f"⚠️ {file} not found!")

print("\n✅ All files downloaded!")


📥 Downloading app.py...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

📥 Downloading run_bot_qdrant.py...


<IPython.core.display.Javascript object>

...

<IPython.core.display.Javascript object>

📥 Downloading retrieval_agent_qdrant.py...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


✅ All files downloaded!


In [14]:
%%writefile requirements.txt
streamlit==1.31.0
pandas==2.1.4
numpy==1.26.3
sentence-transformers==2.3.1
qdrant-client==1.7.0
google-generativeai==0.3.2
requests==2.31.0


Writing requirements.txt


In [15]:
files.download('/content/requirements.txt')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [20]:
%%writefile .dockerignore
__pycache__/
*.pyc
*.pyo
*.pyd
.Python
*.egg-info/
.env
.venv
venv/
*.log
.DS_Store


Overwriting .dockerignore


In [21]:
files.download('/content/.dockerignore')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
%%writefile Dockerfile
# Use Python 3.11 slim image
FROM python:3.11-slim

# Set working directory
WORKDIR /app

# Install system dependencies
RUN apt-get update && apt-get install -y \
    build-essential \
    curl \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first (for caching)
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application files
COPY . .

# Expose port 8080 (Cloud Run requirement)
EXPOSE 8080

# Health check
HEALTHCHECK CMD curl --fail http://localhost:8080/_stcore/health || exit 1

# Run Streamlit
CMD streamlit run app.py \
    --server.port=8080 \
    --server.address=0.0.0.0 \
    --server.headless=true \
    --server.enableCORS=false \
    --server.enableXsrfProtection=false


Writing Dockerfile


In [19]:
files.download('/content/Dockerfile')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>